In [1]:
print("Hello")


Hello


In [2]:
! pip install tensorflow
! pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable


In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [4]:
import os
print("Current Dir",os.getcwd())
curr_dir=os.getcwd()
train_path=os.path.join(curr_dir,'datasets','Train')
testing_path=os.path.join(curr_dir,'datasets','Test')
valid_path=os.path.join(curr_dir,'datasets','Valid')

Current Dir d:\downloads\potato-disease-project


In [5]:
# data importing 
training_set=tf.keras.utils.image_dataset_from_directory(
    train_path,
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    image_size=(128,128),
    shuffle=True,
    interpolation='bilinear'
)

Found 900 files belonging to 3 classes.


In [6]:
training_set.class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [7]:
# data importing 
validation_set=tf.keras.utils.image_dataset_from_directory(
    valid_path,
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    image_size=(128,128),
    shuffle=True,
    interpolation='bilinear'
)

Found 300 files belonging to 3 classes.


In [8]:
cnn=tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',activation='relu',input_shape=[128,128,3]))
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3,padding='same',activation='relu'))
cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=128,kernel_size=3,padding='same',activation='relu'))
cnn.add(tf.keras.layers.Conv2D(filters=128,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=256,kernel_size=3,padding='same',activation='relu'))
cnn.add(tf.keras.layers.Conv2D(filters=256,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=512,kernel_size=3,padding='same',activation='relu'))
cnn.add(tf.keras.layers.Conv2D(filters=512,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

cnn.add(tf.keras.layers.Dropout(0.25))

cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=1500,activation='relu'))
cnn.add(tf.keras.layers.Dropout(0.4))

cnn.add(tf.keras.layers.Dense(units=3,activation='softmax'))


In [9]:
cnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

In [10]:
training_history=cnn.fit(x=training_set,validation_data=validation_set,epochs=10)

Epoch 1/10
29/29 [==============================] - 69s 2s/step - loss: 1.3172 - accuracy: 0.3878 - val_loss: 0.9990 - val_accuracy: 0.5567
Epoch 2/10
29/29 [==============================] - 60s 2s/step - loss: 0.8957 - accuracy: 0.5533 - val_loss: 0.6439 - val_accuracy: 0.6967
Epoch 3/10
29/29 [==============================] - 62s 2s/step - loss: 0.5753 - accuracy: 0.7422 - val_loss: 0.4601 - val_accuracy: 0.8067
Epoch 4/10
29/29 [==============================] - 60s 2s/step - loss: 0.3994 - accuracy: 0.8344 - val_loss: 0.3050 - val_accuracy: 0.8667
Epoch 5/10
29/29 [==============================] - 66s 2s/step - loss: 0.2719 - accuracy: 0.9000 - val_loss: 0.2585 - val_accuracy: 0.8967
Epoch 6/10
29/29 [==============================] - 62s 2s/step - loss: 0.2953 - accuracy: 0.8833 - val_loss: 0.2029 - val_accuracy: 0.9367
Epoch 7/10
29/29 [==============================] - 60s 2s/step - loss: 0.3009 - accuracy: 0.8900 - val_loss: 0.3876 - val_accuracy: 0.8300
Epoch 8/10
29/29 [==

In [13]:
import joblib

# Prepare the dictionary (same as before)
model_data = {
    'architecture': cnn.to_json(),
    'weights': cnn.get_weights()
}

# Save the dictionary to a .joblib file
joblib.dump(model_data, "cnn_model.joblib")


['cnn_model.joblib']

In [14]:
import joblib
from tensorflow.keras.models import model_from_json

# Load the dictionary from the .joblib file
loaded_data = joblib.load("cnn_model.joblib")

# Reconstruct the model from the JSON architecture
loaded_model = model_from_json(loaded_data['architecture'])
loaded_model.set_weights(loaded_data['weights'])

# (Optional) Recompile the model if needed
loaded_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])
